# COPD Deaths - Processing

## Dependencies

Ensure that the required libraries have been installed locally as per the README.md file included in this project.

Run the following cell the import the required dependencies for this notebook.

In [1]:
# Import libraries
import pandas as pd

## Load Dataset

In [2]:
# Load the raw data.
df = pd.read_csv('raw.csv')

## Exploratory Analysis of Raw Data

In [3]:
# Generate summary statistics for object columns.
object_summary_stats = df.describe(include=['O']).transpose()                                                       # Generate summary statistics for object columns.
object_summary_stats['missing_values'] = df.isnull().sum()                                                          # Add missing values to the summary table.
object_summary_stats['present_values'] = df.notnull().sum()                                                         # Add present values to the summary table.
object_summary_stats['datatype'] = df.dtypes                                                                        # Add data types to the summary table.
object_summary_stats = object_summary_stats[['datatype', 'present_values', 'missing_values', 'unique']]             # Select features and reorder table.

# Display the summary tables with titles.
print("Dataset Head:")                                                                                              # Display the dataset head title.
display(df.head().style.set_table_styles([{'selector': 'th', 'props': [('min-width', '100px')]}]))                  # Display the dataset head. For better visualization, set the minimum width of the table headers to 100px.

print("\nObject Summary Statistics:")                                                                               # Display the object summary statistics.
display(object_summary_stats.style.set_table_styles([{'selector': 'th', 'props': [('min-width', '100px')]}]))       # Display the object summary statistics. For better visualization, set the minimum width of the table headers to 100px.

Dataset Head:


,Risk group,Sex,LHD,Period,"Rate per 100,000 population",LL 95% CI,UL 95% CI
0,Older adults (65+ years),Males,Sydney LHD,2011-2012,192.200000,158.800000,230.400000
1,Older adults (65+ years),Males,Sydney LHD,2012-2013,199.800000,166.300000,237.900000
2,Older adults (65+ years),Males,Sydney LHD,2013-2014,227.200000,192.100000,266.800000
3,Older adults (65+ years),Males,Sydney LHD,2014-2015,212.000000,178.800000,249.600000
4,Older adults (65+ years),Males,Sydney LHD,2015-2016,164.400000,135.700000,197.300000



Object Summary Statistics:


,datatype,present_values,missing_values,unique
Risk group,object,1920,0,2
Sex,object,1920,0,3
LHD,object,1920,0,16
Period,object,1920,0,10


## Data Manipulation

In [4]:
# Rename columns to match Air Quality data set.
df = df.rename(columns={'LHD': 'lhd', 'Period': 'financial year'})                                                  # Rename columns
df.columns = df.columns.str.lower()                                                                                 # Lowercase column names

# Remove ' LHD' for Local Health District values.
df['lhd'] = df['lhd'].str.replace(' LHD', '')                                                                       # Remove ' LHD' from lhd column

# Remove rows representing state-wide aggregated data.
df = df.dropna(subset=['lhd'])                                                                                      # Remove rows with missing values in 'lhd' column
df = df[~df['lhd'].str.contains('All')]                                                                             # Remove rows with 'All' in 'lhd' column

# Remove columns holding Confidence Interval data.
df = df.loc[:, ~df.columns.str.contains('% ci')]                                                                    # Remove columns with '% ci' in the name

# Reformat 'financial year' values from XX/YY to XXXX/YYYY.
df['financial year'] = df['financial year'].str.replace('-', '/')                                                   # Replace '-' with '/'

# Remove rows where 'risk group' is not 'All ages'.
df = df[df['risk group'] == 'All ages']                                                                             # Keep rows where 'risk group' is 'All ages'
df = df.drop(columns=['risk group'])                                                                                # Drop 'risk group' column

# View the cleaned data.
display(df.head())                                                                                                  # Display the first few rows of the cleaned data

,sex,lhd,financial year,"rate per 100,000 population"
480,Males,Sydney,2011/2012,26.5
481,Males,Sydney,2012/2013,27.8
482,Males,Sydney,2013/2014,31.1
483,Males,Sydney,2014/2015,29.5
484,Males,Sydney,2015/2016,23.3


In [5]:
# Pivot the dataframe to have 'sex' as columns
df = df.pivot_table(index=['financial year', 'lhd'], columns='sex', values='rate per 100,000 population').reset_index()   # Pivot the dataframe

# Rename the columns to match the desired format
df.columns.name = None                                                                                                    # Remove the columns name 
df = df.rename(columns={                                                                                                  # Rename columns
    'Persons': 'Persons rate per 100,000 population',
    'Males': 'Male rate per 100,000 population', 
    'Females': 'Female rate per 100,000 population'
})

# View the DataFrame
display(df.head()) 

,financial year,lhd,"Female rate per 100,000 population","Male rate per 100,000 population","Persons rate per 100,000 population"
0,2011/2012,Central Coast,24.9,33.1,28.2
1,2011/2012,Far West,30.9,38.5,32.6
2,2011/2012,Hunter New England,19.1,34.3,25.4
3,2011/2012,Illawarra Shoalhaven,20.3,34.0,26.1
4,2011/2012,Mid North Coast,18.8,38.8,27.7


## Set Date Range

Set the range of financial years from 2014/2015 to 2023/2024.

In [6]:
# Drop pre 2014/2015 data.
df = df[~df['financial year'].isin(['2011/2012', '2012/2013', '2013/2014'])]                                        # Drop pre 2014/2015 data.

# Add rows for each LHD for the missing years until 2023/2024.
missing_rows = []                                                                                                   # Create a list to store the missing rows.
lhds = df['lhd'].unique()                                                                                           # Get unique LHDs.
years = [f"{year}/{year + 1}" for year in range(2014, 2024)]                                                        # Create a list of years from 2014/2015 to 2023/2024.

for lhd in lhds:
    for year in years:
        if not ((df['lhd'] == lhd) & (df['financial year'] == year)).any():                                         # Check if the row is missing.
            missing_rows.append({                                                                                   # Append the missing row to the list.
                'lhd': lhd, 
                'financial year': year, 
                'Female rate per 100,000 population': None,
                'Male rate per 100,000 population': None,
                'Persons rate per 100,000 population': None,
            })   

# Create a DataFrame from the missing rows and concatenate it to the original DataFrame
if missing_rows:
    df_missing = pd.DataFrame(missing_rows)                                                                         # Create a DataFrame from the missing rows.
    df = pd.concat([df, df_missing], ignore_index=True)                                                             # Concatenate the DataFrames.

# Sort the DataFrame by 'lhd' and 'financial year'.
df = df.sort_values(by=['lhd', 'financial year']).reset_index(drop=True)                                            # Sort the DataFrame by 'lhd' and 'financial year'.

# View the DataFrame.
display(df.tail())                                                                                                  # View the last 5 rows of the DataFrame.

C:\Users\matth\AppData\Local\Temp\ipykernel_10520\1010056951.py:23: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([df, df_missing], ignore_index=True)                                                             # Concatenate the DataFrames.


,financial year,lhd,"Female rate per 100,000 population","Male rate per 100,000 population","Persons rate per 100,000 population"
145,2019/2020,Western Sydney,13.5,24.5,18.3
146,2020/2021,Western Sydney,13.8,20.7,16.8
147,2021/2022,Western Sydney,NaN,NaN,NaN
148,2022/2023,Western Sydney,NaN,NaN,NaN
149,2023/2024,Western Sydney,NaN,NaN,NaN


Fill missing values using linear interpolation.

In [7]:
df['Female rate per 100,000 population'] = df['Female rate per 100,000 population'].interpolate()                                    # Fill missing values using linear interpolation.
df['Male rate per 100,000 population'] = df['Male rate per 100,000 population'].interpolate()                                        # Fill missing values using linear interpolation.
df['Persons rate per 100,000 population'] = df['Persons rate per 100,000 population'].interpolate()                                  # Fill missing values using linear interpolation.

## Output Processed Dataset

In [8]:
# Save the processed data to a new CSV file.
df.to_csv('processed.csv', index=False)                                                                            # Save the processed data to a new CSV file.                                                                                                      # View the first 5 rows of the DataFrame.

## View Processed Dataset

In [9]:
# Display the processed data.
display(df.head())

,financial year,lhd,"Female rate per 100,000 population","Male rate per 100,000 population","Persons rate per 100,000 population"
0,2014/2015,Central Coast,25.3,43.1,33.0
1,2015/2016,Central Coast,28.2,40.0,33.1
2,2016/2017,Central Coast,33.1,39.6,35.5
3,2017/2018,Central Coast,33.9,40.6,36.4
4,2018/2019,Central Coast,31.6,41.1,35.4
